In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


In [74]:
def extract_relation_names(json_data):
    relation_names = set()
    
  # Using set to ensure unique values
    for item in json_data:
        
        
        extract_relation_names_from_plan(item, relation_names)
    return relation_names

def extract_relation_names_from_plan(plan, relation_names):
    relation_name = plan.get("Relation Name")
    if relation_name:
        relation_names.add(relation_name)
    if "Plans" in plan:
        for subplan in plan["Plans"]:
            extract_relation_names_from_plan(subplan, relation_names)


In [85]:
def extract_filters(data):
    filters = []

    if isinstance(data, dict):
        for key, value in data.items():
            if key == "Filter":
                filters.append(value)
            elif isinstance(value, (dict, list)):
                filters.extend(extract_filters(value))

    elif isinstance(data, list):
        for item in data:
            filters.extend(extract_filters(item))

    return filters

In [29]:
df = pd.read_json('new.json')



In [27]:
columns =['Plan','query','avg_row_length','no_of_partitions','Execution Time']

In [ ]:
df_copy = df[columns]


In [80]:
relation_names=[]
for row in df_copy['Plan']:
    row = json.dumps(row)
    # print(row)
    relation_name = extract_relation_names([json.loads(row)])
    relation_names.append(relation_name)

{'customer', 'orders'}


In [ ]:
df_copy['relation_names'] = relation_names
df_copy.head(10)

In [93]:
filters=[]
for row in df_copy['Plan']:
    row = json.dumps(row)
    
    filter = extract_filters(json.loads(row))
    filters.append(filter)

[["(l_shipdate <= '1998-08-12 00:00:00'::timestamp without time zone)"], ["(l_shipdate <= '1998-08-12 00:00:00'::timestamp without time zone)"], ["(l_shipdate <= '1998-08-12 00:00:00'::timestamp without time zone)"], ["(l_shipdate <= '1998-08-12 00:00:00'::timestamp without time zone)"], ["(l_shipdate <= '1998-08-12 00:00:00'::timestamp without time zone)"], ["(l_shipdate <= '1998-08-12 00:00:00'::timestamp without time zone)"], ["(l_shipdate <= '1998-08-12 00:00:00'::timestamp without time zone)"], ["(l_returnflag = 'R'::bpchar)"], ["(n_name = 'EGYPT'::bpchar)", '(sum((partsupp.ps_supplycost * (partsupp.ps_availqty)::numeric)) > $2)', "(n_name = 'EGYPT'::bpchar)"], ["((l_shipmode = ANY ('{RAIL,AIR}'::bpchar[])) AND (l_commitdate < l_receiptdate) AND (l_shipdate < l_commitdate) AND (l_receiptdate >= '1996-01-01'::date) AND (l_receiptdate < '1997-01-01 00:00:00'::timestamp without time zone))"], ["((o_comment)::text !~~ '%special%accounts%'::text)"], [], ['(NOT (hashed SubPlan 1))', "((

In [96]:
filter_len = []
for i in filters:
    filter_len.append(len(i))

df_copy['filters'] = filters
df_copy['filter_len'] = filter_len  

In [97]:
df_copy.head(10)

,Plan,query,avg_row_length,no_of_partitions,relation_names,filters,filter_len
0,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",[156.32257463759336],[0],{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1
1,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",[156.32257463759336],[0],{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1
2,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",[156.32257463759336],[0],{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1
3,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",[156.32257463759336],[0],{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1
4,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",[156.32257463759336],[0],{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1
5,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",[156.32257463759336],[0],{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1
6,"{'Node Type': 'Limit', 'Parallel Aware': False...","select\n\tl_returnflag,\n\tl_linestatus,\n\tsu...",[156.32257463759336],[0],{lineitem},[(l_shipdate <= '1998-08-12 00:00:00'::timesta...,1
7,"{'Node Type': 'Limit', 'Parallel Aware': False...","\nselect\n\tc_custkey,\n\tc_name,\n\tsum(l_ext...","[141.49479813333335, 156.32257463759336, 139.1...","[0, 0, 0, 0]","{orders, nation, customer, lineitem}",[(l_returnflag = 'R'::bpchar)],1
8,"{'Node Type': 'Limit', 'Parallel Aware': False...","\n\nselect\n\tps_partkey,\n\tsum(ps_supplycost...","[177.41824475, 139.16, 178.07392]","[0, 0, 0]","{partsupp, nation, supplier}","[(n_name = 'EGYPT'::bpchar), (sum((partsupp.ps...",3
9,"{'Node Type': 'Limit', 'Parallel Aware': False...","\n\nselect\n\tl_shipmode,\n\tsum(case\n\t\twhe...","[141.49479813333335, 156.32257463759336]","[0, 0]","{orders, lineitem}","[((l_shipmode = ANY ('{RAIL,AIR}'::bpchar[])) ...",1
